In [ ]:
## AGENDA
## - Shortes path algorithms within NEO4j

In [ ]:
#from py2neo import Graph
#graph = Graph("bolt://neo4j.sktai.io:30074", auth=("neo4j","!neo4j00"))

In [ ]:
# https://github.com/neo4j-contrib/neo4j-apoc-procedures/releases/tag/3.5.0.1
#mv ./apoc-3.5.0.1-all.jar $Neo4J_HOME/plugins

In [1]:
from neo4j import GraphDatabase
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [2]:
#gph_conn = Neo4jConnection(uri="bolt://neo4j.sktai.io:30074", user="neo4j", pwd="!neo4j00")
#gph_conn = Neo4jConnection(uri="bolt://localhost:7687", user="neo4j", pwd="!neo4j00")
gph_conn = Neo4jConnection(uri="bolt://127.0.0.1:7687", user="neo4j", pwd="!neo4j00")

In [3]:
gph_conn.query("""
               CALL db.schema();
               """)

[<Record nodes=[] relationships=[]>]

In [4]:
gph_conn.query("""
          MATCH (n)
          DETACH DELETE n
          """)

[]

In [ ]:
## SET DATA

In [5]:
gph_conn.query("""
               CREATE (A:Node {name: "A"})
               CREATE (B:Node {name: "B"})
               CREATE (C:Node {name: "C"})
               CREATE (D:Node {name: "D"})
               CREATE (E:Node {name: "E"})
               
               CREATE (A)-[:LINKED_TO {weight: 10}]->(B)
               CREATE (A)-[:LINKED_TO {weight: 33}]->(C)
               CREATE (A)-[:LINKED_TO {weight: 35}]->(D)
               CREATE (B)-[:LINKED_TO {weight: 20}]->(C)
               CREATE (C)-[:LINKED_TO {weight: 28}]->(D)
               CREATE (C)-[:LINKED_TO {weight: 6 }]->(E)
               CREATE (D)-[:LINKED_TO {weight: 40}]->(E)
               """)

[]

In [ ]:
## PROJECT GRAPH

In [6]:
gph_conn.query("""
               CALL gds.graph.create("graph", "Node", "LINKED_TO")
               """)

[<Record graphName='graph' nodeProjection={'Node': {'properties': {}, 'label': 'Node'}} relationshipProjection={'LINKED_TO': {'orientation': 'NATURAL', 'aggregation': 'DEFAULT', 'type': 'LINKED_TO', 'properties': {}}} nodeCount=5 relationshipCount=7 createMillis=33>]

In [ ]:
## Dijkstra's

In [8]:
gph_conn.query("""
               MATCH (A:Node {name: "A"})
               MATCH (E:Node {name: "E"})
               CALL gds.alpha.shortestPath.stream("graph", {startNode: A, endNode: E})
               YIELD nodeId, cost 
               RETURN gds.util.asNode(nodeId).name as name, cost
               """)

[<Record name='A' cost=0.0>,
 <Record name='D' cost=1.0>,
 <Record name='E' cost=2.0>]

In [9]:
gph_conn.query("""
               MATCH (A:Node {name: "A"})
               MATCH (E:Node {name: "E"})
               CALL gds.alpha.shortestPath.stream("graph", {startNode: A, endNode: E, relationshipWeightProperty: "weight"})
               YIELD nodeId, cost 
               RETURN gds.util.asNode(nodeId).name as name, cost
               """)

Query failed: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `gds.alpha.shortestPath.stream`: Caused by: java.lang.IllegalArgumentException: Relationship weight property `weight` not found in graph with relationship properties: []}


In [10]:
gph_conn.query("""
               CALL gds.graph.list("graph") 
               YIELD relationshipProjection 
               RETURN *
               """)

[<Record relationshipProjection={'LINKED_TO': {'orientation': 'NATURAL', 'aggregation': 'DEFAULT', 'type': 'LINKED_TO', 'properties': {}}}>]

In [11]:
gph_conn.query("""
               CALL gds.graph.create("graph_weighted", "Node", "LINKED_TO", { relationshipProperties: [{weight: 'weight' }] } )
               """)

[<Record graphName='graph_weighted' nodeProjection={'Node': {'properties': {}, 'label': 'Node'}} relationshipProjection={'LINKED_TO': {'orientation': 'NATURAL', 'aggregation': 'DEFAULT', 'type': 'LINKED_TO', 'properties': {'weight': {'property': 'weight', 'defaultValue': nan, 'aggregation': 'DEFAULT'}}}} nodeCount=5 relationshipCount=7 createMillis=16>]

In [12]:
gph_conn.query("""
               CALL gds.graph.list("graph_weighted") 
               YIELD relationshipProjection 
               RETURN *
               """)

[<Record relationshipProjection={'LINKED_TO': {'orientation': 'NATURAL', 'aggregation': 'DEFAULT', 'type': 'LINKED_TO', 'properties': {'weight': {'property': 'weight', 'defaultValue': nan, 'aggregation': 'DEFAULT'}}}}>]

In [13]:
gph_conn.query("""
               MATCH (A:Node {name: "A"})
               MATCH (E:Node {name: "E"})
               CALL gds.alpha.shortestPath.stream("graph_weighted", {startNode: A, endNode: E, relationshipWeightProperty: "weight"})
               YIELD nodeId, cost 
               RETURN gds.util.asNode(nodeId).name as name, cost
               """)

[<Record name='A' cost=0.0>,
 <Record name='B' cost=10.0>,
 <Record name='C' cost=30.0>,
 <Record name='E' cost=36.0>]

In [ ]:
## ADD PATHs

In [14]:
gph_conn.query("""
               MATCH (A:Node {name: "A"})
               MATCH (C:Node {name: "C"})
               MATCH (E:Node {name: "E"})
               
               CREATE (C)-[:LINKED_TO {weight: 20}]->(A)
               CREATE (E)-[:LINKED_TO {weight: 5}]->(C)
               """)

[]